In [25]:
csvfile = r'C:\Users\roumba\Documents\Software\deep-events\Videos\Video_1\Brightfield\labels1b.csv'

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io
import plotly.express as px
from scipy import ndimage as ndi
import tensorflow
import imageio
from skimage.segmentation import flood, flood_fill
from scipy import signal
import tensorflow_probability as tfp
import tensorflow as tf

datacsv = pd.read_csv(csvfile)
#data.columns


In [2]:
def get_gaussian(mu, sigma, size):
    mu = ((mu[1]+0.5-0.5*size[1])/(size[1]*0.5), (mu[0]+0.5-0.5*size[0])/(size[0]*0.5))
    sigma = (sigma[0]/size[0], sigma[1]/size[1])
    mvn = tfp.distributions.MultivariateNormalDiag(loc=mu, scale_diag=sigma)
    x,y = tf.cast(tf.linspace(-1,1,size[0]),tf.float32), tf.cast(tf.linspace(-1,1,size[1]), tf.float32)
    # meshgrid as a list of [x,y] coordinates
    coords = tf.reshape(tf.stack(tf.meshgrid(x,y),axis=-1),(-1,2)).numpy()
    gauss = mvn.prob(coords)
    return tf.reshape(gauss, size)


In [17]:
sigma_trial=(20,20)
size_trial=(2048,2048)

In [54]:
points_of_interest= np.zeros((100,img.height,img.width))

for row_number in range (0, len(datacsv)):
    framenumber_in_row= int(datacsv.loc[row_number, 'axis-0'])
    fission_ycoord = int(datacsv.loc[row_number, 'axis-1'])
    fission_xcoord = int(datacsv.loc[row_number, 'axis-2'])
    fission_coords=(fission_ycoord,fission_xcoord)
    gaussian_points=get_gaussian(fission_coords,sigma_trial,size_trial)
    for areay_replacement in range (0,20):
        for areax_replacement in range (0,20):
            pointy_replacement=fission_ycoord+areay_replacement
            pointx_replacement=fission_xcoord+areax_replacement
            pointyy_replacement=fission_ycoord-areay_replacement
            pointxx_replacement=fission_xcoord-areax_replacement
            points_of_interest[framenumber_in_row,pointy_replacement,pointx_replacement]=gaussian_points[pointy_replacement,pointx_replacement]
            points_of_interest[framenumber_in_row,pointyy_replacement,pointxx_replacement]=gaussian_points[pointyy_replacement,pointxx_replacement]
            points_of_interest[framenumber_in_row,pointy_replacement,pointxx_replacement]=gaussian_points[pointy_replacement,pointxx_replacement]
            points_of_interest[framenumber_in_row,pointyy_replacement,pointx_replacement]=gaussian_points[pointyy_replacement,pointx_replacement]
            

imageio.mimwrite('points.tiff', points_of_interest)

In [20]:
def event_separation(data):
    #this function takes in the data from the excel file and splits them into a nested list: each list within the nested list corresponds to the excel lines of a single division 
    #it differentiates the lines based on conditions on both frame number and x,y-distance that could potentially be changed if they don't work
    length_of_file=len(data)-1
    all_event_lines=[]
    single_divison_events=[]
    for excelindex in range(0,length_of_file):
        framedataline1= data.iloc[excelindex,1] 
        framedataline2= data.iloc[excelindex+1,1] 
        framediff= abs(framedataline2-framedataline1)

        ydistancedataline1= data.iloc[excelindex,2]
        ydistancedataline2= data.iloc[excelindex+1,2]
        ydistancediff= abs(ydistancedataline2-ydistancedataline1)

        xdistancedataline1= data.iloc[excelindex,2]
        xdistancedataline2= data.iloc[excelindex+1,2]
        xdistancediff= abs(xdistancedataline2-xdistancedataline1)

        if framediff < 3 and ydistancediff <  10 and xdistancediff < 10 :
            single_divison_events.append(excelindex)
            if excelindex == length_of_file-1:
                single_divison_events.append(excelindex+1)
                all_event_lines.append(single_divison_events)
        else:
            single_divison_events.append(excelindex)
            all_event_lines.append(single_divison_events)
            single_divison_events=[]
    return all_event_lines

In [21]:
def image_crop_save(l,list, data, img):   
    division_list=[]
    for index_list in range(0, l):
        l1=len(list[index_list])
        division_list=[]

        for index_list1 in range(0,l1):
            division_list.append(list[index_list][index_list1])
            minlist=division_list[0]
            maxlist=division_list[index_list1]

        data_croped= data.iloc[minlist:maxlist,1:4]
        frame1=int(data_croped['axis-0'].min())
        frame2=int(data_croped['axis-0'].max())
        ymean = data_croped['axis-1'].mean()
        xmean = data_croped['axis-2'].mean()
        ycrop1=ymean+128
        ycrop2=ymean-128
        xcrop1=xmean+128
        xcrop2=xmean-128
        dataar=np.zeros((frame2-frame1, 256, 256))

        for frame_index, frame_number in enumerate(range (frame1, frame2)):
            img.seek(frame_number-1) #starts from 0 I think?
            box = (xcrop2, ycrop2, xcrop1, ycrop1) #choose dimensions of box
            imcrop= img.crop(box)
        
            dataar[frame_index, :, :] = np.array(imcrop)
            
        currname = f'image_{index_list}.tiff'
        imageio.mimwrite(currname, dataar)
    return(dataar) #what does it actually return

In [55]:
list1=event_separation(datacsv)
l=len(list1)
#gauss=r'C:\Users\roumba\Documents\Software\deep-events\points.tiff'
#gauss_image=Image.open(gauss)
#image_crop_save(l,list1, datacsv, gauss_image)